In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.model_selection import KFold
from statsmodels.tsa.seasonal import seasonal_decompose

<h1>Hyperparameters<h1>


In [149]:
k = 5 # validation folds
min_lag = 120
max_lag = 148

<h1>Creating DataFrames<h1>

In [3]:
spot_prices = pd.read_csv("spotpriser.csv").set_index("time")
consumption = pd.read_csv("consumption_temp_with_flags.csv").set_index("time")

oslo_consumption = consumption[consumption["location"] == "oslo"]
oslo_consumption = oslo_consumption.merge(spot_prices["oslo"], left_index=True, right_index=True, how="left")
oslo_consumption=oslo_consumption.drop(['location'],axis=1).rename(columns={'oslo':'spot_price'})

stavanger_consumption = consumption[consumption["location"] == "stavanger"]
stavanger_consumption = stavanger_consumption.merge(spot_prices["stavanger"], left_index=True, right_index=True, how="left")
stavanger_consumption=stavanger_consumption.drop(['location'],axis=1).rename(columns={'stavanger':'spot_price'})

trondheim_consumption = consumption[consumption["location"] == "trondheim"]
trondheim_consumption = trondheim_consumption.merge(spot_prices["trondheim"], left_index=True, right_index=True, how="left")
trondheim_consumption=trondheim_consumption.drop(['location'],axis=1).rename(columns={'trondheim':'spot_price'})

tromso_consumption = consumption[consumption["location"] == "tromsø"]
tromso_consumption = tromso_consumption.merge(spot_prices["tromsø"], left_index=True, right_index=True, how="left")
tromso_consumption=tromso_consumption.drop(['location'],axis=1).rename(columns={'tromsø':'spot_price'})

bergen_consumption = consumption[consumption["location"] == "bergen"]
bergen_consumption = bergen_consumption.merge(spot_prices["bergen"], left_index=True, right_index=True, how="left")
bergen_consumption=bergen_consumption.drop(['location'],axis=1).rename(columns={'bergen':'spot_price'})
    

<h1>Feature engineering<h1>

In [ ]:
CONSUMPTION_DFS = [oslo_consumption,stavanger_consumption,trondheim_consumption,tromso_consumption,bergen_consumption]

def decompose_time_series(df, period=24):
    for col in columns:
        # Check if the column exists in the DataFrame
        if col in df.columns:
            # Decompose the time series
            result = seasonal_decompose(df[col], model='additive', period=period)

            # Create new columns for trend and seasonal components
            df[f"{col}_trend"] = result.trend
            df[f"{col}_seasonal"] = result.seasonal

            # Drop the original column
            df.drop(col, axis=1, inplace=True)
        else:
            print(f"Column '{col}' not found in DataFrame.")
    
    return df

In [4]:
CONSUMPTION_DFS = [oslo_consumption,stavanger_consumption,trondheim_consumption,tromso_consumption,bergen_consumption]

cols_to_standard_normal = ["temperature","delta_temp"]
cols_to_normalize_01 = ["consumption","spot_price"]

for df in CONSUMPTION_DFS:
    df["is_holiday"] = df["is_holiday"].astype(int)
    df["is_weekend"] = df["is_weekend"].astype(int)
    df.index = pd.to_datetime(ALL_DFS[i].index)
    df["hour"] = ALL_DFS[i].index.hour


    for col in cols_to_standard_normal:
        mu, sig = df[col].mean(), df[col].std()
        df[col] = (df[col]-mu)/sig
    for col in cols_to_normalize_01:
        df[col] = (df[col]-df[col].min())/(df[col].max()-df[col].min())
    df = df.iloc[1:,:]


def add_delta_columns(df,exclude_columns, min_mag,max_lag):
    for col in df.columns:
        if col not in exclude_columns:
            for delta in range(min_mag,max_lag+1):
                df[f"{col}_L{delta}"] =  



<h1>Splitting datasets<h1>

In [5]:
oslo_with_price = oslo_consumption.dropna()
oslo_no_price = pd.concat([oslo_consumption,oslo_with_price]).drop_duplicates(keep=False).drop(columns=["spot_price"])

stavanger_with_price = stavanger_consumption.dropna()
stavanger_no_price = pd.concat([stavanger_consumption,stavanger_with_price]).drop_duplicates(keep=False).drop(columns=["spot_price"])

trondheim_with_price = trondheim_consumption.dropna()
trondheim_no_price = pd.concat([trondheim_consumption,trondheim_with_price]).drop_duplicates(keep=False).drop(columns=["spot_price"])

tromso_with_price = tromso_consumption.dropna()
tromso_no_price = pd.concat([tromso_consumption,tromso_with_price]).drop_duplicates(keep=False).drop(columns=["spot_price"])

bergen_with_price = bergen_consumption.dropna()
bergen_no_price = pd.concat([bergen_consumption,bergen_with_price]).drop_duplicates(keep=False).drop(columns=["spot_price"])



In [153]:
oslo_with_price

,consumption,temperature,is_holiday,is_weekend,spot_price,delta_temp
time,,,,,,
2022-10-11 00:00:00,0.212632,-0.307026,0,0,0.178110,-0.326610
2022-10-11 01:00:00,0.202627,-0.318428,0,0,0.170600,-0.108693
2022-10-11 02:00:00,0.193783,-0.318428,0,0,0.158543,0.000265
2022-10-11 03:00:00,0.207115,-0.295624,0,0,0.161547,0.218182
2022-10-11 04:00:00,0.246200,-0.261418,0,0,0.176637,0.327139
...,...,...,...,...,...,...
2023-04-02 17:00:00,0.659003,-0.215810,0,1,0.146333,-1.525151
2023-04-02 18:00:00,0.686003,-0.295624,0,1,0.155992,-0.762443
2023-04-02 19:00:00,0.695320,-0.774508,0,1,0.167787,-4.575982


In [154]:
ALL_DFS = [oslo_with_price,oslo_no_price,stavanger_with_price,stavanger_no_price,trondheim_with_price,trondheim_no_price,tromso_with_price,tromso_no_price,bergen_with_price,bergen_no_price]


/var/folders/js/9hpr3tt17xd6ttv9xy3zrhw80000gn/T/ipykernel_51030/2434831662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ALL_DFS[i]["hour"] = ALL_DFS[i].index.hour
/var/folders/js/9hpr3tt17xd6ttv9xy3zrhw80000gn/T/ipykernel_51030/2434831662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ALL_DFS[i]["hour"] = ALL_DFS[i].index.hour
/var/folders/js/9hpr3tt17xd6ttv9xy3zrhw80000gn/T/ipykernel_51030/2434831662.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

<h1>Making training data<h1>

In [155]:
# Seed for reproducibility
np.random.seed(0)

data=oslo_with_price

# Function to create dataset with lags for multiple variables
def create_multivariate_dataset(data, min_lag=1, max_lag=3):
    X, y = [], []
    for i in range(max_lag, len(data)):
        lagged_data = data.iloc[i-max_lag:i-min_lag+1, 1:]  # Shape: [time_steps, num_predictors]
        X.append(lagged_data.values)
        y.append(data.iloc[i, 0])  # Target variable (Sales)
    return np.array(X), np.array(y)

# Hyperparameter ranges
min_lag_values = [120]
max_lag_values = [156,180]
lstm_units_values = [15,60]

# k-Fold for validation
k = 5
kf = KFold(n_splits=k, shuffle=False)

# Grid Search
best_mape = float('inf')
best_hyperparams = None

print("Min Max Units || MAPE")
for min_lag in min_lag_values:
    for max_lag in max_lag_values:
        for lstm_units in lstm_units_values:
            if max_lag > min_lag:
                print(f"{min_lag}   {max_lag}   {lstm_units}   ||",end="")
                X, y = create_multivariate_dataset(data, min_lag, max_lag)
                mape_values = []

                for train_index, val_index in kf.split(X):
                    X_train, X_val = X[train_index], X[val_index]
                    y_train, y_val = y[train_index], y[val_index]

                    # Building and Training the LSTM Model
                    model = Sequential()
                    model.add(LSTM(lstm_units, activation='relu', input_shape=(X.shape[1], X.shape[2])))
                    model.add(Dense(1))
                    model.compile(optimizer='adam', loss='mse')

                    # Training the model
                    model.fit(X_train, y_train, epochs=5, verbose=0)

                    # Making predictions
                    y_pred = model.predict(X_val, verbose=0).flatten()

                    # Calculating MAPE for the fold
                    mape = np.mean(np.abs((y_val - y_pred) / y_val)) * 100
                    mape_values.append(mape)

                # Averaging MAPE across all folds
                average_mape = np.mean(mape_values)
                print(f" {average_mape}")
                # Update best hyperparameters if current MAPE is lower
                if average_mape < best_mape:
                    best_mape = average_mape
                    best_hyperparams = (min_lag, max_lag, lstm_units)

print(f'Best Hyperparameters: min_lag={best_hyperparams[0]}, max_lag={best_hyperparams[1]}, LSTM Units={best_hyperparams[2]}')
print(f'Best MAPE: {best_mape}%')

# Store the best model's predictions and actual values
best_predictions = []
best_actuals = []

# Retrain the best model
X, y = create_multivariate_dataset(data, *best_hyperparams[:2])
kf = KFold(n_splits=k, shuffle=False)

for train_index, val_index in kf.split(X):
    X_train, X_val = X[train_index], X[val_index]
    y_train, y_val = y[train_index], y[val_index]

    # Building and Training the LSTM Model with best hyperparameters
    model = Sequential()
    model.add(LSTM(best_hyperparams[2], activation='relu', input_shape=(X.shape[1], X.shape[2])))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    # Training the model
    model.fit(X_train, y_train, epochs=5, verbose=0)

    # Making predictions
    y_pred = model.predict(X_val, verbose=0).flatten()

    best_predictions.extend(y_pred)
    best_actuals.extend(y_val)

# Plot the predictions against the actual values
plt.figure(figsize=(10, 6))
plt.plot(best_actuals, label='Actual Sales')
plt.plot(best_predictions, label='Predicted Sales', color='red')
plt.title('Best Model Predictions vs Actual Sales')
plt.xlabel('Time')
plt.ylabel('Sales')
plt.legend()
plt.show()




Min Max Units || MAPE
120   156   25   || 30.34829300077933
120   156   50   || 26.769534680807574
120   156   75   || 25.623734807642972
120   168   25   || 36.12610933663527
120   168   50   || 32.026647355166595
120   168   75   || 66.68879974730233
120   180   25   || 27.04526908109431
120   180   50   || 24.57236350421365
120   180   75   || 58338268.57595573
120   400   25   || nan
120   400   50   ||

KeyboardInterrupt: 